In [ ]:
import requests
from obi_auth import get_token
from entitysdk import Client, ProjectContext, models
from enum import Enum
import obi_one as obi

class DatabaseEnvironment(Enum):
    STAGING_DATABASE = "staging"
    # PRODUCTION_DATABASE = "production"
    # LOCAL_UNAUTHENTICATED = "local_unauthenticated"
    # Note: LOCAL_AUTHENTICATED is not supported by entitycore because the project/vlab ids do not exist in your keycloak groups

obi_one_api_url = "http://127.0.0.1:8100"
# obi_one_api_url = "https://staging.openbraininstitute.org/api/obi-one"

add_morphology_asset = False

database_environment = DatabaseEnvironment.STAGING_DATABASE
# database_environment = DatabaseEnvironment.PRODUCTION_DATABASE
# database_environment = DatabaseEnvironment.LOCAL_UNAUTHENTICATED

if database_environment == DatabaseEnvironment.STAGING_DATABASE:
    """
    - Uncomment in obi-one/.env.run-local before "make run-local": 
        export ENTITYCORE_URL=https://staging.openbraininstitute.org/api/entitycore
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"
    virtual_lab_id=obi.LAB_ID_STAGING_TEST
    project_id=obi.PROJECT_ID_STAGING_TEST

# elif database_environment == DatabaseEnvironment.PRODUCTION_DATABASE:
#     """
#     For future: EntityCore not currently in production.
#     - Uncomment in obi-one/.env.run-local before "make run-local": 
#         export ENTITYCORE_URL=https://www.openbraininstitute.org/api/entitycore
#     """
#     entitycore_api_url = "https://www.openbraininstitute.org/api/entitycore"

elif database_environment == DatabaseEnvironment.LOCAL_UNAUTHENTICATED:
    """
    Not yet tested.
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk)
    """
    entitycore_api_url = "http://127.0.0.1:8000"
    add_morphology_asset = True
    virtual_lab_id=""
    project_id=""
    
else:
    raise ValueError(f"Unsupported environment: {database_environment}")


token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

# Circuit ID

In [ ]:
circuit_id = "3d0dcdfd-0534-49e6-aeea-d78d35948c7f"

# Create SimulationForm using Cirucit

In [ ]:


# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(campaign_name="O1 Simulation", campaign_description="Simulation of O1 circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Neuron Sets
sim_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(10)))
sim_conf.add(sim_neuron_set, name='L1All')

all_neuron_set = obi.AllNeurons()
sim_conf.add(all_neuron_set, name='AllNeurons')

sync_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet2", elements=range(3)))
sim_conf.add(sync_neuron_set, name='ID3')

vpm_neuron_set = obi.nbS1VPMInputs(sample_percentage=100)
sim_conf.add(vpm_neuron_set, name='nbS1VPMInputs')

# Regular Timesteps
regular_timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=sim_duration)
sim_conf.add(regular_timestamps, name='RegularTimestamps')

# Stimulus
poisson_input = obi.PoissonSpikeStimulus(duration=800.0, timestamps=regular_timestamps.ref, frequency=20, source_neuron_set=sim_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
sim_conf.add(poisson_input, name='PoissonInputStimulus')

sync_input = obi.FullySynchronousSpikeStimulus(timestamps=regular_timestamps.ref, source_neuron_set=sync_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
sim_conf.add(sync_input, name='SynchronousInputStimulus')

current_stim = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timestamps.ref, neuron_set=sim_neuron_set.ref, amplitude=0.1)
sim_conf.add(current_stim, name='ConstantCurrentClampSomaticStimulus')

# Recordings
voltage_recording = obi.SomaVoltageRecording(neuron_set=sim_neuron_set.ref)
sim_conf.add(voltage_recording, name='VoltageRecording')

time_window_voltage_recording = obi.TimeWindowSomaVoltageRecording(neuron_set=sim_neuron_set.ref, start_time=0.0, end_time=2000.0)
sim_conf.add(time_window_voltage_recording, name='TimeWindowVoltageRecording')

# Initialization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=obi.CircuitFromID(id_str=circuit_id), 
                                                        node_set=sim_neuron_set.ref, 
                                                        simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

# Generate GridScan (locally) - Note strange issue described for relative paths

In [ ]:
import tempfile

# Create a temporary directory (not used in this example, but can be useful for testing)
with tempfile.TemporaryDirectory() as temp_dir:

    """
    Very strange issue with adding assets when a relative path is used here. Need to look into after CNS.
    """
    # grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../obi-output/circuit_simulations/grid_scan')
    grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='/Users/james/Documents/obi/code/obi-output/circuit_simulations/grid_scan')
    # grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root=temp_dir)
    grid_scan.multiple_value_parameters(display=True)
    grid_scan.coordinate_parameters(display=True)
    grid_scan.execute(processing_method='generate', data_postprocessing_method='save', db_client=client)

# Test endpoint. Service must be launched first (make run-local)

In [ ]:
import requests

# Construct the full endpoint URL
url = f"{obi_one_api_url}/generated/simulations-generate-grid-save"
# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Construct request
# Placeholder example:
request_body = {
 'type': 'SimulationsForm',
 'timestamps': {'RegularTimestamps': {'type': 'RegularTimestamps',
   'start_time': 0.0,
   'number_of_repetitions': 3,
   'interval': 3000.0}},
 'stimuli': {
  'PoissonInputStimulus': {'type': 'PoissonSpikeStimulus',
   'timestamps': {'type': 'TimestampsReference',
    'block_dict_name': 'timestamps',
    'block_name': 'RegularTimestamps'},
   'source_neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1All'},
   'targeted_neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1All'},
   'duration': 800.0,
   'random_sample': 0,
   'frequency': 20.0}
   },
 'recordings': {'VoltageRecording': {'type': 'SomaVoltageRecording',
   'start_time': 0.0,
   'end_time': 3000.0,
   'dt': 0.1,
   'neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1All'}}},
 'neuron_sets': {'L1All': {'type': 'IDNeuronSet',
    'sample_percentage': 100,
   'sample_seed': 0,
   'neuron_ids': {'type': 'NamedTuple',
    'name': 'IDNeuronSet1',
    'elements': (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)}}},
 'initialize': {'type': 'SimulationsForm.Initialize',
  'circuit': {'type': 'CircuitFromID',
   'id_str': circuit_id},
  'simulation_length': 3000.0,
  'node_set': {'type': 'NeuronSetReference',
   'block_dict_name': 'neuron_sets',
   'block_name': 'L1All'},
  'sample_seed': 1,
  'extracellular_calcium_concentration': 1.1,
  'v_init': -80.0},
 'info': {'type': 'Info',
  'campaign_name': 'No name provided',
  'campaign_description': 'No description provided'}
}

# Make the POST request
response = requests.post(url, headers=headers, json=request_body)

# Check the response
if response.status_code == 200:
    data = response.json()
    simulation_campaign_id = data
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

# Stage the Circuit

In [ ]:
from entitysdk.models import Circuit
circuit = client.get_entity(entity_id=circuit_id, entity_type=Circuit)

from entitysdk.staging.circuit import stage_circuit
from pathlib import Path
stage_circuit(client=client, model=circuit, output_dir=Path('temp_circ/'))

# Stage the Simulation

In [ ]:
from entitysdk.models import SimulationCampaign, Simulation
simulation_campaign = client.get_entity(entity_id=simulation_campaign_id, entity_type=SimulationCampaign)
simulation = client.get_entity(entity_id=simulation_campaign.simulations[0].id, entity_type=Simulation)

for asset in simulation.assets:
    if asset.content_type == "application/json":
        if asset.label == 'sonata_simulation_config':
            # Download the content into memory
            content = client.download_file(
                entity_id=simulation.id,
                entity_type=Simulation,
                asset_id=asset.id,
                output_path='simulation_config.json'
            )

from pathlib import Path
from entitysdk.staging.simulation import stage_simulation
stage_simulation(client=client, model=simulation, output_dir=Path('temp_sim'), circuit_config_path=Path("../temp_circ/circuit_config.json"))